In [3]:
import os
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# from sklearn.externals import joblib
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
dataset_path = "../dataset/train/"

In [ ]:
# Load image paths and corresponding labels
image_paths = []
labels = []

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image_paths.append(image_path)
            labels.append(class_name)

In [ ]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# Load and preprocess images
images = []
for image_path in image_paths:
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Preprocess image for VGG16
    images.append(img_array)

In [ ]:
X = np.array(images)
y = np.array(labels_encoded)

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Flatten the image data for SVM
X_train_flatten = X_train.reshape(X_train.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

In [ ]:
# Train an SVM
svm_model = svm.SVC(kernel="linear")
svm_model.fit(X_train_flatten, y_train)

: 

: 